In [1]:
%use adventOfCode

In [2]:
val aoc = AocClient.fromEnv().interactiveDay(2025, 9)
aoc.viewPartOne()

--- Day 9: Movie Theater --- 
 You slide down the firepole in the corner of the playground and land in the North Pole base movie theater! 
 The movie theater has a big tile floor with an interesting pattern. Elves here are redecorating the theater by switching out some of the square tiles in the big grid they form. Some of the tiles are red ; the Elves would like to find the largest rectangle that uses red tiles for two of its opposite corners. They even have a list of where the red tiles are located in the grid (your puzzle input). 
 For example: 
 7,1
11,1
11,7
9,7
9,5
2,5
2,3
7,3
 
 Showing red tiles as # and other tiles as . , the above arrangement of red tiles would look like this: 
 ..............
.......#...#..
..............
..#....#......
..............
..#......#....
..............
.........#.#..
..............
 
 You can choose any two red tiles as the opposite corners of your rectangle; your goal is to find the largest rectangle possible. 
 For example, you could make a rectangle (shown as O ) with an area of 24 between 2,5 and 9,7 : 
 ..............
.......#...#..
..............
..#....#......
..............
.. O OOOOOOO....
..OOOOOOOO....
..OOOOOOO O .#..
..............
 
 Or, you could make a rectangle with area 35 between 7,1 and 11,7 : 
 ..............
....... O OOOO..
.......OOOOO..
..#....OOOOO..
.......OOOOO..
..#....OOOOO..
.......OOOOO..
.......OOOO O ..
..............
 
 You could even make a thin rectangle with an area of only 6 between 7,3 and 2,3 : 
 ..............
.......#...#..
..............
.. O OOOO O ......
..............
..#......#....
..............
.........#.#..
..............
 
 Ultimately, the largest rectangle you can make in this example has area 50 . One way to do this is between 2,5 and 11,1 : 
 ..............
..OOOOOOOOO O ..
..OOOOOOOOOO..
..OOOOOOOOOO..
..OOOOOOOOOO..
.. O OOOOOOOOO..
..............
.........#.#..
..............
 
 Using two red tiles as opposite corners, what is the largest area of any rectangle you can make?

In [28]:
sealed class Tile(open val x: Int, open val y: Int)

data class RedTile(override val x: Int, override val y: Int) : Tile(x, y)
data class GreenTile(override val x: Int, override val y: Int) : Tile(x, y)

typealias Rectangle = Pair<RedTile, RedTile>

fun rectangleSizeOf(corner1: RedTile, corner2: RedTile) =
    (maxOf(corner1.x, corner2.x) - minOf(corner1.x, corner2.x) + 1).toLong() *
        (maxOf(corner1.y, corner2.y) - minOf(corner1.y, corner2.y) + 1).toLong()

fun rectangleSizeOf(cornerPair: Rectangle) = rectangleSizeOf(cornerPair.first, cornerPair.second)

fun Rectangle.shrinkOrNull(): Rectangle? {
    val (corner1, corner2) = this
    val minX = minOf(corner1.x, corner2.x) + 1
    val minY = minOf(corner1.y, corner2.y) + 1
    val maxX = maxOf(corner1.x, corner2.x) - 1
    val maxY = maxOf(corner1.y, corner2.y) - 1

    return if (maxX < minX || maxY < minY) {
        null
    } else {
        RedTile(minX, minY) to RedTile(maxX, maxY)
    }
}

operator fun Rectangle.contains(tile: Tile): Boolean =
    tile.x in minOf(first.x, second.x)..maxOf(first.x, second.x) &&
        tile.y in minOf(first.y, second.y)..maxOf(first.y, second.y)

data class Floor(
    val redTiles: List<RedTile>,
    // pt2
    val greenBorderTiles: List<GreenTile> = emptyList(),
) {

    val carthesianProduct: Sequence<Rectangle> =
        redTiles.indices.asSequence().flatMap { i ->
            redTiles.indices.asSequence().mapNotNull { j ->
                if (i == j) return@mapNotNull null
                minOf(i, j) to maxOf(i, j)
            }
        }.distinct()
            .map { (i, j) -> redTiles[i] to redTiles[j] }

    val border = redTiles + greenBorderTiles

    fun lines(): Sequence<String> = sequence {
        val minX = redTiles.minOf { it.x } - 1
        val maxX = redTiles.maxOf { it.x } + 1
        val minY = redTiles.minOf { it.y } - 1
        val maxY = redTiles.maxOf { it.y } + 1

        val grid = Array(maxY - minY + 1) { ByteArray(maxX - minX + 1) { 0 } }

        for (redTile in redTiles) grid[redTile.y - minY][redTile.x - minX] = 1
        for (greenTile in greenBorderTiles) grid[greenTile.y - minY][greenTile.x - minX] = 2
        for (row in grid) {
            yield(
                row.joinToString("") {
                    when (it) {
                        0.toByte() -> "."
                        1.toByte() -> "#"
                        2.toByte() -> "X"
                        else -> error("Invalid tile value $it")
                    }
                }
            )
        }
    }

    override fun toString(): String = lines().joinToString("\n")
}

fun String.parse() = Floor(this.trim().lines().map { it.split(",").let { (x, y) -> RedTile(x.toInt(), y.toInt()) } })
fun String.parsePt2(): Floor {
    val floor = parse()
    val greenBorderTiles = floor.redTiles.plus(floor.redTiles.first())
        .zipWithNext()
        .flatMap { (a, b) ->
            when {
                a.x == b.x -> ((minOf(a.y, b.y) + 1)..(maxOf(a.y, b.y) - 1)).map { GreenTile(a.x, it) }
                a.y == b.y -> ((minOf(a.x, b.x) + 1)..(maxOf(a.x, b.x) - 1)).map { GreenTile(it, a.y) }
                else -> error("Tiles $a and $b are not adjacent")
            }
        }
    return floor.copy(greenBorderTiles = greenBorderTiles)
}

In [29]:
val exampleInput = """7,1
11,1
11,7
9,7
9,5
2,5
2,3
7,3"""

val exampleFloor = exampleInput.parse()
exampleFloor

............
......#...#.
............
.#....#.....
............
.#......#...
............
........#.#.
............

In [5]:
exampleFloor.carthesianProduct.maxBy { rectangleSizeOf(it) }
    .let {
        rectangleSizeOf(it)
    }

50

In [6]:
val input = aoc.input().parse()
val res = input.carthesianProduct.maxOf(::rectangleSizeOf)
res

4755064176

In [7]:
//aoc.submitPartOne(res)

In [8]:
aoc.viewPartTwo()

--- Part Two --- 
 The Elves just remembered: they can only switch out tiles that are red or green . So, your rectangle can only include red or green tiles. 
 In your list, every red tile is connected to the red tile before and after it by a straight line of green tiles . The list wraps, so the first red tile is also connected to the last red tile. Tiles that are adjacent in your list will always be on either the same row or the same column. 
 Using the same example as before, the tiles marked X would be green: 
 ..............
.......#XXX#..
.......X...X..
..#XXXX#...X..
..X........X..
..#XXXXXX#.X..
.........X.X..
.........#X#..
..............
 
 In addition, all of the tiles inside this loop of red and green tiles are also green. So, in this example, these are the green tiles: 
 ..............
.......#XXX#..
.......XXXXX..
..#XXXX#XXXX..
..XXXXXXXXXX..
..#XXXXXX#XX..
.........XXX..
.........#X#..
..............
 
 The remaining tiles are never red nor green. 
 The rectangle you choose still must have red tiles in opposite corners, but any other tiles it includes must now be red or green. This significantly limits your options. 
 For example, you could make a rectangle out of red and green tiles with an area of 15 between 7,3 and 11,1 : 
 ..............
.......OOOO O ..
.......OOOOO..
..#XXXX O OOOO..
..XXXXXXXXXX..
..#XXXXXX#XX..
.........XXX..
.........#X#..
..............
 
 Or, you could make a thin rectangle with an area of 3 between 9,7 and 9,5 : 
 ..............
.......#XXX#..
.......XXXXX..
..#XXXX#XXXX..
..XXXXXXXXXX..
..#XXXXXX O XX..
.........OXX..
......... O X#..
..............
 
 The largest rectangle you can make in this example using only red and green tiles has area 24 . One way to do this is between 9,5 and 2,3 : 
 ..............
.......#XXX#..
.......XXXXX..
.. O OOOOOOOXX..
..OOOOOOOOXX..
..OOOOOOO O XX..
.........XXX..
.........#X#..
..............
 
 Using two red tiles as opposite corners, what is the largest area of any rectangle you can make using only red and green tiles?

In [30]:
val exampleFloor2 = exampleInput.parsePt2()
exampleFloor2

............
......#XXX#.
......X...X.
.#XXXX#...X.
.X........X.
.#XXXXXX#.X.
........X.X.
........#X#.
............

In [10]:
aoc.input().parsePt2().lines().forEach(::println) // too large to print lol

java.lang.OutOfMemoryError: Java heap space

Let's check all boxes... shrinking them inward once...
and then check whether any of the bounding tiles are inside the box?

In [31]:
val floor = aoc.input().parsePt2()
floor.redTiles.size + floor.greenBorderTiles.size

590702

In [32]:
val largestRect = exampleFloor2.carthesianProduct.maxBy { rect ->
    val shrunkRect = rect.shrinkOrNull() ?: return@maxBy 0L
    val cutsBorder = exampleFloor2.border.any { it in shrunkRect }
    if (cutsBorder) 0L else rectangleSizeOf(rect)
}

largestRect to rectangleSizeOf(largestRect)

((RedTile(x=9, y=7), RedTile(x=2, y=5)), 24)

In [33]:
val largestRect = floor.carthesianProduct.maxBy { rect ->
    val shrunkRect = rect.shrinkOrNull() ?: return@maxBy 0L
    val cutsBorder = floor.border.any { it in shrunkRect }
    if (cutsBorder) 0L else rectangleSizeOf(rect)
}

largestRect to rectangleSizeOf(largestRect)

((RedTile(x=6165, y=68497), RedTile(x=94671, y=50270)), 1613305596)

In [34]:
//aoc.submitPartTwo(rectangleSizeOf(largestRect))

Your answer: 1613305596. 
 That's the right answer! You are one gold star closer to decorating the North Pole. 
 You have completed Day 9! You can [Share on Bluesky Twitter Mastodon ] this victory or [Return to Your Advent Calendar] .